In [1]:
# импортируем нужные библиотеки 
import pandas as pd
import datetime as dt

In [55]:
# загрузим файл с данными
data = pd.read_csv('2019-Dec.csv')
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-12-01 00:00:00 UTC,remove_from_cart,5712790,1487580005268456287,NaN,f.o.x,6.27,576802932,51d85cb0-897f-48d2-918b-ad63965c12dc
1,2019-12-01 00:00:00 UTC,view,5764655,1487580005411062629,NaN,cnd,29.05,412120092,8adff31e-2051-4894-9758-224bfa8aec18
2,2019-12-01 00:00:02 UTC,cart,4958,1487580009471148064,NaN,runail,1.19,494077766,c99a50e8-2fac-4c4d-89ec-41c05f114554
3,2019-12-01 00:00:05 UTC,view,5848413,1487580007675986893,NaN,freedecor,0.79,348405118,722ffea5-73c0-4924-8e8f-371ff8031af4
4,2019-12-01 00:00:07 UTC,view,5824148,1487580005511725929,NaN,NaN,5.56,576005683,28172809-7e4a-45ce-bab0-5efa90117cd5


In [56]:
# возьмем информацию только по просмотрам и покупкам пользователей
data = data[(data.event_type == 'purchase') | (data.event_type == 'view')]

# оставим столбцы: время, тип действия, айди продукта, айди пользователя
data = data[['event_time', 'event_type', 'product_id', 'user_id']].reset_index(drop=True)

# поменяем формат даты и оставим только дату без времени
data.event_time = pd.to_datetime(data.event_time)
data.event_time = data.event_time.apply(lambda x: x.date())

data.head()

,event_time,event_type,product_id,user_id
0,2019-12-01,view,5764655,412120092
1,2019-12-01,view,5848413,348405118
2,2019-12-01,view,5824148,576005683
3,2019-12-01,view,5773361,560109803
4,2019-12-01,view,5807805,576005683


In [57]:
# создадим две отдельные таблицы соответсвенно действиям пользователей  

# просмотр товара
# также удалим повторяющиеся строчки, потому что пользователь может несколько раз зайти на страничку с товаром
df_view = data[['event_time', 'product_id', 'user_id']][data['event_type'] == 'view'].drop_duplicates()
df_view.rename(columns = {'event_time':'date of view', 'product_id':'product_id', 'user_id': 'user_id'}, inplace = True )
# покупка товара
df_buy = data[['event_time', 'product_id', 'user_id']][data['event_type'] == 'purchase']
df_buy.rename(columns = {'event_time':'date of bought', 'product_id':'product_id', 'user_id': 'user_id'}, inplace = True )


# соединим две таблицу по двум ключам: айди товара и айди пользователя
df = df_view.merge(df_buy, on=['product_id', 'user_id'])
df.drop_duplicates(subset=['date of bought', 'product_id', 'user_id'], inplace=True)
df.drop_duplicates(subset=['date of view', 'product_id', 'user_id'], inplace=True)
# если пользователь только посмотрел товар, но не купил, удаляем его из таблицы
df = df.drop(df[(df['date of bought']-df['date of view']) < dt.timedelta(0)].index)

In [60]:
# создадим таблицу уже непосредственно для когортного анализа
# добавим дату просмотора товара
df_cohort = pd.DataFrame([])
df_cohort['date of view'] = df['date of view']
# и столбец с разницей в днях между просмотром и покупкой товара
df_cohort['day of bought after view'] = df['date of bought'] - df['date of view']

# добавим последний столбец
# 'count of view-buy' - кол-во уникальных сочетаний пользователь-продукт 
# для определенной даты просмотра продукта и кол-ва дней после просмотра, когда он купил этот товар
# цифра 1 означает, что пользователь купил товар, а так как каждая строчка таблицы содержит дату с датой покупки
# то везде ставим 1 
df_cohort['count of view-buy'] = 1
df_cohort = df_cohort.groupby(['date of view', 'day of bought after view']).agg(total_bought=('count of view-buy', 'sum')).reset_index(drop=False)

In [61]:
df_cohort.to_excel('cosmentics.xlsx')

In [ ]:
# d = pd.read_csv('2019-Dec.csv')
# d[(d.user_id == 572081598) & (d.product_id == 5809910)]

In [69]:
df_view[df_view['date of view'] == '2019-12-01']

,date of view,product_id,user_id


In [70]:
df_view.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1397937 entries, 0 to 1941506
Data columns (total 3 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   date of view  1397937 non-null  object
 1   product_id    1397937 non-null  int64 
 2   user_id       1397937 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 42.7+ MB
